In [ ]:
!pip install pandas-profiling==2.7.1

In [ ]:
import pandas as pd
import math
from pandas_profiling import ProfileReport

In [ ]:
def get_distance_two_locations(lat1, lon1, lat2, lon2):
    rad=math.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia

In [ ]:
def get_sum_of_poi_client(row, poi_df):
  total_poi = 0.0
  total_habitantes = 0.0
  secciones_censales_lista = []
  for _, r in poi_df.iterrows():
    distancia = get_distance_two_locations(row['ord_billing_address_lat'],row['ord_billing_address_long'], r['latitude'], r['longitude'])
    if distancia < 0.5:
      row[r['type']] = row[r['type']] + 1.0
      total_poi = total_poi + 1.0
  row['total_poi'] = total_poi
  return row

In [ ]:
jeff_vlc = pd.read_csv('dataset_jeff_valencia.csv', sep=',', decimal='.')

In [ ]:
poi_vlc = pd.read_csv('poi_tratado.csv', sep=',', decimal='.')

In [ ]:
jeff_vlc['Dist_Cliente_Tienda'] = jeff_vlc.apply(lambda row: get_distance_two_locations(row['prtn_address_latitude'],row['prtn_address_longitude'],row['ord_billing_address_lat'],row['ord_billing_address_long']), axis=1)

In [ ]:
uniquevalues = poi_vlc['type'].unique()
jeff_vlc[uniquevalues] = 0.0

In [ ]:
jeff_vlc = jeff_vlc.apply(lambda row: get_sum_of_poi_client(row,poi_vlc), axis=1)

In [ ]:
jeff_vlc_petxina=jeff_vlc[jeff_vlc['prtn_code']=="PETXINA"]
jeff_vlc_reino=jeff_vlc[jeff_vlc['prtn_code']=="REINO"]

In [ ]:
report = ProfileReport(jeff_vlc_petxina)
report.to_file("jeff_vlc_petxina_report.html")

In [ ]:
report = ProfileReport(jeff_vlc_reino)
report.to_file("jeff_vlc_reino_report.html")

In [ ]:
report = ProfileReport(jeff_vlc)
report.to_file("jeff_vlc_report.html")

In [ ]:
jeff_vlc.to_csv(r'jeff_vlc_poi.csv')